In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Let's try to do our best with a few ML model

In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [4]:
sklearn.__version__

'0.18.1'

In [5]:
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)

from sklearn.cross_validation import KFold

C:\Users\Delanoue\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.

C:\Users\Delanoue\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



## Data Preparation

We use the data preparation explained in the notebook called "Data Preparation for our studies "


import pandas as pd
import numpy as np
from transplant.config import *

df = pd.read_csv(PATH_STATIC_CLEAN)[STATIC_CATEGORIES['patient_intraoperative']]
declampage_col = list(df.filter(like='declampage_cote_2').columns)
declampage_col.append('id_patient')

X_intra=df[declampage_col].apply(pd.to_numeric, errors='coerce').dropna(1, how="all")

from transplant.data.dataset import Dataset
dataset = Dataset()

train, test = dataset.get_static()
X_train=train[['id_patient', 'target']]
X_test=test[['id_patient', 'target']]

X_train_intra_with_targ=pd.merge(X_train, X_intra, on='id_patient')
X_test_intra_with_targ=pd.merge(X_test, X_intra, on='id_patient')

mean_train = X_train_intra_with_targ.mean()

train_filled = X_train_intra_with_targ.fillna(mean_train)
test_filled = X_test_intra_with_targ.fillna(mean_train)



X_train = np.array(train_filled.drop(['target','id_patient'], axis=1))
X_test = np.array(test_filled.drop(['target','id_patient'], axis=1))
y_train = np.array(train_filled['target'])
y_test = np.array(test_filled['target'])
X_col = train_filled.drop(['target','id_patient'], axis=1).columns

In [6]:
def armand_funct():
    """
    How to : X_train, X_test, y_train ,Y_test, col = armand_funct()
    """
    from transplant.data.dataset import Dataset
    
    
    # To get Target
    dataset = Dataset()

    train, test = dataset.get_static()
    train['is_train'] = 1
    test['is_train'] = 0
    all_stat = pd.concat([train, test])
    all_stat = all_stat[['id_patient', 'target', 'is_train']].copy()
    
    # Intra
    df = pd.read_csv(PATH_STATIC_CLEAN)[STATIC_CATEGORIES['patient_intraoperative']]
    declampage = list(df.filter(like='declampage_cote_2').columns)
    declampage.append('id_patient')
    df = df[declampage]
    
    # Intra Num
    df_num = df.apply(pd.to_numeric, errors='coerce')
    
    # Merge with all_statto get target 
    df_num = df_num.merge(all_stat, on='id_patient')
    
    # Feature engi
    df_num['nb_nan_feature'] = pd.isnull(df_num).sum(axis=1)
    
    df_num = df_num.fillna(-99) # 
    
    
    # Split 
    
    y_train = np.array(df_num[df_num['is_train'] == 1]['target'])
    y_test = np.array(df_num[df_num['is_train'] == 0]['target'])
    
    X_train = df_num[df_num['is_train'] == 1].drop(['target', 'id_patient', 'is_train'], axis=1)
    X_test = df_num[df_num['is_train'] == 0].drop(['target', 'id_patient', 'is_train'], axis=1)
    col = X_train.columns
    
    return X_train.values, X_test.values, y_train ,y_test, col

In [7]:
from transplant.data.dataset import Dataset
dataset=Dataset()

In [8]:
from transplant.data.learningset import Learningset
from transplant.config import *
learningset = Learningset()

#X_train, X_test, y_train, y_test, X_col = learningset.get_static_filled(to_train=True)
#X_train, X_test, y_train, y_test, X_col = learningset.get_data_merged_dynamic_flatten_full("cls", False)
X_train, X_test, y_train, y_test, X_col = armand_funct()

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((287, 24), (123, 24), (287,), (123,))

We need to change the y format in order to make our models work

In [10]:
y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((287, 24), (123, 24), (287, 1), (123, 1))

## ML Tuning

In [12]:
cols = X_col  # On a besoin des colonnes pour plus tard
# On observe le grand X et pas le petit x ...

ntrain = X_train.shape[0]

ntest = X_test.shape[0]

seed_fix = 5                   # Pour reproduire les résultats

In [13]:
ntrain, ntest

(287, 123)

In [14]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)

    def fit(self, x, y):
        return self.clf.fit(x, y)

    def feature_importances(self, x, y):
        return self.clf.fit(x, y).feature_importances_

In [15]:
NFOLDS = 5
kf = KFold(ntrain, n_folds=NFOLDS, random_state=seed_fix)


def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [16]:
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    'warm_start': True,
    # 'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'verbose': 0
}

In [17]:
et_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    # 'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

In [18]:
ada_params = {
    'n_estimators': 500,
    'learning_rate': 0.75
}

In [19]:
gb_params = {
    'n_estimators': 500,
    # 'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

In [20]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=seed_fix, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=seed_fix, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=seed_fix, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier,
                   seed=seed_fix, params=gb_params)

In [21]:
X_train.shape, y_train.shape, X_test.shape

((287, 24), (287, 1), (123, 24))

## Learning zone

In [22]:
%%time
et_oof_train, et_oof_test = get_oof(
    et, X_train, y_train, X_test)  # Extra Trees
rf_oof_train, rf_oof_test = get_oof(
    rf, X_train, y_train, X_test)  # Random Forest
ada_oof_train, ada_oof_test = get_oof(
    ada, X_train, y_train, X_test)  # AdaBoost
gb_oof_train, gb_oof_test = get_oof(
    gb, X_train, y_train, X_test)  # Gradient Boost

print("I am done learning :D")

I am done learning :D
Wall time: 26.9 s


In [23]:
rf_feature = rf.feature_importances(X_train, y_train)
et_feature = et.feature_importances(X_train, y_train)
ada_feature = ada.feature_importances(X_train, y_train)
gb_feature = gb.feature_importances(X_train, y_train)

# On met plutot dans un dataFrame
feature_dataframe = pd.DataFrame({'features': cols,
                                  'Random Forest Importance des features': rf_feature,
                                  'Extra Trees Importance des features': et_feature,
                                  'AdaBoost Importance des features': ada_feature,
                                  'Gradient Boost Importance des features': gb_feature
                                  })

In [24]:
rf_pred = rf.predict(X_test)
et_pred = et.predict(X_test)
ada_pred = ada.predict(X_test)
gb_pred = gb.predict(X_test)

## Classification Reports

For the ones who forgot : 

Compute the precision

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The best value is 1 and the worst value is 0.

Compute the recall

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The best value is 1 and the worst value is 0.

The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:

F1 = 2 (precision recall) / (precision + recall)

The support is the number of occurrences of each class in y_true

In [25]:
from sklearn.metrics import confusion_matrix

In [26]:
confusion_matrix(y_test, rf_pred)

array([[60, 12],
       [31, 20]])

In [27]:
confusion_matrix(y_test, et_pred)

array([[65,  7],
       [42,  9]])

In [28]:
confusion_matrix(y_test, ada_pred)

array([[53, 19],
       [27, 24]])

In [29]:
confusion_matrix(y_test, gb_pred)

array([[47, 25],
       [27, 24]])

In [30]:
from sklearn import metrics
print(metrics.classification_report(rf_pred, y_test))

             precision    recall  f1-score   support

          0       0.83      0.66      0.74        91
          1       0.39      0.62      0.48        32

avg / total       0.72      0.65      0.67       123



In [31]:
print(metrics.classification_report(et_pred, y_test))

             precision    recall  f1-score   support

          0       0.90      0.61      0.73       107
          1       0.18      0.56      0.27        16

avg / total       0.81      0.60      0.67       123



In [32]:
print(metrics.classification_report(ada_pred, y_test))

             precision    recall  f1-score   support

          0       0.74      0.66      0.70        80
          1       0.47      0.56      0.51        43

avg / total       0.64      0.63      0.63       123



In [33]:
print(metrics.classification_report(gb_pred, y_test))

             precision    recall  f1-score   support

          0       0.65      0.64      0.64        74
          1       0.47      0.49      0.48        49

avg / total       0.58      0.58      0.58       123



## Feature Importance Visualisation 

In [34]:
trace = go.Scatter(x=feature_dataframe['features'].values,
                   y=feature_dataframe['Random Forest Importance des features'].values,

                   mode='markers',
                   marker=dict(
    sizemode='diameter',
    sizeref=1,
    size=25,
    # color = np.random.randn(500), #set color equal to a variable
    color=feature_dataframe['Random Forest Importance des features'].values,
    colorscale='Portland',
    showscale=True
),
    text=feature_dataframe['features'].values
)

data = [trace]

layout = go.Layout(
    autosize=True,
    title='Random Forest Importance des features',
    hovermode='closest',
    yaxis=dict(
        title='Importance des features',
        ticklen=5,
        gridwidth=2
    ),
    showlegend=False
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='scatter2010')

In [35]:
trace = go.Scatter(
    y=feature_dataframe['Extra Trees Importance des features'].values,
    x=feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=1,
        size=25,
        color=feature_dataframe['Extra Trees Importance des features'].values,
        colorscale='Portland',
        showscale=True
    ),
    text=feature_dataframe['features'].values
)
data = [trace]

layout = go.Layout(
    autosize=True,
    title='Extra Trees Importance des features',
    hovermode='closest',

    yaxis=dict(
        title='Importance des features',
        ticklen=5,
        gridwidth=2
    ),
    showlegend=False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='scatter2010')

In [36]:
trace = go.Scatter(
    y=feature_dataframe['AdaBoost Importance des features'].values,
    x=feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=1,
        size=25,
        color=feature_dataframe['AdaBoost Importance des features'].values,
        colorscale='Portland',
        showscale=True
    ),
    text=feature_dataframe['features'].values
)
data = [trace]

layout = go.Layout(
    autosize=True,
    title='AdaBoost Importance des features',
    hovermode='closest',
    yaxis=dict(
        title='Importance des features',
        ticklen=5,
        gridwidth=2
    ),
    showlegend=False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='scatter2010')

In [37]:
trace = go.Scatter(
    y=feature_dataframe['Gradient Boost Importance des features'].values,
    x=feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=1,
        size=25,
        color=feature_dataframe['Gradient Boost Importance des features'].values,
        colorscale='Portland',
        showscale=True
    ),
    text=feature_dataframe['features'].values
)
data = [trace]

layout = go.Layout(
    autosize=True,
    title='Gradient Boosting Importance des features',
    hovermode='closest',
    yaxis=dict(
        title='Importance des features',
        ticklen=5,
        gridwidth=2
    ),
    showlegend=False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='scatter2010')